# Debug

In [3]:
import re
from collections import defaultdict
import pandas as pd

# Percorso al file di log
log_file_path = "/data/users/etosato/ANM_Verona/src/cnn/runs/run1/log_train1"

# Pattern regex
epoch_pattern = re.compile(r"\[VAL DEBUG - EPOCH (\d+)\]")
debug_pattern = re.compile(
    r"\[VAL DEBUG\] ID: ([\w_]+) \| True: (\d) \| Pred: (\d) \| Prob: \[([\d.e\-]+), ([\d.e\-]+)\]"
)

# Variabili temporanee
current_epoch = None
parsed_data = []

# Parsing del file
with open(log_file_path, "r") as f:
    for line in f:
        epoch_match = epoch_pattern.match(line)
        debug_match = debug_pattern.match(line)

        if epoch_match:
            current_epoch = int(epoch_match.group(1))
        elif debug_match and current_epoch is not None:
            subj_id = debug_match.group(1)
            true_label = int(debug_match.group(2))
            pred_label = int(debug_match.group(3))
            prob0 = float(debug_match.group(4))
            prob1 = float(debug_match.group(5))

            parsed_data.append({
                "epoch": current_epoch,
                "ID": subj_id,
                "true": true_label,
                "pred": pred_label,
                "prob_0": prob0,
                "prob_1": prob1,
            })

# Creazione DataFrame
df_debug = pd.DataFrame(parsed_data)

# Conteggio frequenza per soggetto
id_counts = df_debug["ID"].value_counts().reset_index()
id_counts.columns = ["ID", "count"]


In [5]:
id_counts

,ID,count
0,1_S_5101,4
1,1_S_5093,4
2,1_S_5030,4
3,1_S_5058,4
4,1_S_5074,4
...,...,...
79,1_S_5096,1
80,1_S_5062,1
81,305_S_6498,1
82,1_S_5064,1


# Utils

In [1]:
import pandas as pd

# === File paths ===
base_dir = "/data/users/etosato/ANM_Verona/src/cnn/tuning_results"
csv_pairs = [
    ("adni_psp.csv", "all_testing_results_ADNI_PSP.csv"),
    ("adni_cbs.csv", "all_testing_results_ADNI_CBS.csv"),
    ("psp_cbs.csv", "all_testing_results_PSP_CBS.csv"),
]

merged_dfs = []

for tuning_file, results_file in csv_pairs:
    # Load CSVs
    df_tuning = pd.read_csv(f"{base_dir}/{tuning_file}")
    df_results = pd.read_csv(f"{base_dir}/{results_file}")

    # Drop duplicate 'group' column from tuning
    df_tuning = df_tuning.drop(columns=["group"])

    # Merge on ['tuning', 'config']
    df_merged = pd.merge(df_results, df_tuning, on=["tuning", "config"], how="left")

    # Save or collect for further usage
    merged_dfs.append(df_merged)

    # Optionally: save merged result to new CSV
    merged_filename = results_file.replace("all_testing_results_", "merged_")
    df_merged.to_csv(f"{base_dir}/{merged_filename}", index=False)


# Augmentation check

In [1]:
import random
import pandas as pd
import os

In [ ]:
n_augmentations = 10
subset_size = 17

# Path to augmented FC maps directory
fcmaps_augmented_dir = '/Users/emmatosato/Documents/PhD/ANM_Verona/data/FCmaps_augmented'

# List of 173 HCP subjects
hcp_list_path = '/Users/emmatosato/Documents/PhD/ANM_Verona/data_utils/debugging/list_HCP.txt'

# Output txt report
path_final_report = "/Users/emmatosato/Documents/PhD/ANM_Verona/data_utils/debugging/final_report.txt"

# Path to CSV file with missing SCA files
path_sca_missing = "/Users/emmatosato/Documents/PhD/ANM_Verona/data_utils/debugging/missing_SCA_files.csv"

# Summary CSV path of augmentation info
csv_aug_path = '/Users/emmatosato/Documents/PhD/ANM_Verona/data_utils/metadata/aug_tracking.csv'

## HCP List and Subsets

Check if the method used in augmentation.py works (note: it already has checks inside)

In [ ]:
# Read HCP file
with open(hcp_list_path, "r") as f:
    hcp_pool = [line.strip() for line in f.readlines()]

random.seed(42)

# Generate fixed non-overlapping HCP subsets
shuffled_hcp = random.sample(hcp_pool, n_augmentations * subset_size)

hcp_subsets = [
    shuffled_hcp[i * subset_size : (i + 1) * subset_size]
    for i in range(n_augmentations)
]

Check that each subset has exactly 17 elements

In [ ]:
for idx, subset in enumerate(hcp_subsets, start=1):
    if len(subset) != subset_size:
        print(f"Subset {idx} has incorrect length: {len(subset)} elements")
    else:
        print(f"Subset {idx} OK ({len(subset)} HCP)")

Check which HCPs were left out

In [ ]:
used_hcp = set(shuffled_hcp)
hcp_pool_set = set(hcp_pool)
excluded_hcp = hcp_pool_set - used_hcp

print("List of excluded HCPs:", sorted(excluded_hcp))

Check for overlaps between subsets

In [ ]:
# Flatten all subsets into a single list
all_hcps = [hcp for subset in hcp_subsets for hcp in subset]

# Compare total length vs. number of unique HCPs
if len(all_hcps) == len(set(all_hcps)):
    print("Subsets are disjoint: no overlaps.")
else:
    print("Warning: there are overlapping HCPs between subsets.")

## Augmentation Check

#### CSV

In [ ]:
aug_track = pd.read_csv(csv_aug_path)

In [ ]:
aug_track.iloc[:5, :5]

Check that there are exactly 177 unique subjects

In [ ]:
unique_subjects = aug_track['subject'].unique()
print(f"Number of unique subjects: {len(unique_subjects)}")

Check that each subject has 10 augmentations

In [ ]:
augment_per_subject = aug_track.groupby("subject")["augmentation"].count()
subjects_missing = augment_per_subject[augment_per_subject != 10]

if subjects_missing.empty:
    print("All subjects have exactly 10 augmentations.")
else:
    print("Some subjects DO NOT have 10 augmentations")
    print(subjects_missing)

Check that each augmentation has exactly 17 HCPs for all patients

In [ ]:
# Verify that each row in the DataFrame has exactly 17 HCPs
aug_track["hcp_count"] = aug_track["hcp_subset"].apply(lambda x: len(x.split(",")))

# Filter rows with fewer or more than 17 HCPs
invalid_hcp_counts = aug_track[aug_track["hcp_count"] != 17]

# Output
if invalid_hcp_counts.empty:
    print("All augmentations have exactly 17 HCPs.")
else:
    print("Some augmentations do NOT have 17 HCPs")

Excluding inconsistent subjects, check that valid ones use the same HCPs per augmentation index across all subjects.
That is:
- **Sub1**
    - *aug 1*: hcp1, hcp2, hcp3
    - *aug 2*: hcp4, hcp5, hcp6
    -  ...
...
- **Sub2**
    - *aug 1*: hcp1, hcp2, hcp3
    - *aug 2*: hcp4, hcp5, hcp6
    -  ...

In [ ]:
# Exclude subjects with incomplete augmentations
valid_aug_track = aug_track[aug_track["hcp_count"] == 17].copy()

# For each augmentation index, compute the most frequent HCP group
ref_hcp_by_aug = valid_aug_track.groupby("augmentation")["hcp_subset"].agg(lambda x: x.mode().iloc[0])

# Assign expected value to each row
valid_aug_track["expected_hcp_subset"] = valid_aug_track["augmentation"].map(ref_hcp_by_aug)

# Find inconsistent subjects
df_inconsistent = valid_aug_track[valid_aug_track["hcp_subset"] != valid_aug_track["expected_hcp_subset"]]

# Final output
if df_inconsistent.empty:
    print("Same HCPs used for each augmentation across all VALID subjects.")
else:
    print("Some augmentations have different HCPs between subjects (only among valid ones):")
    display(df_inconsistent[["subject", "augmentation", "hcp_subset", "expected_hcp_subset"]].sort_values(["augmentation", "subject"]))


Subjects with inconsistencies

In [ ]:
print("Subjects with inconsistencies in HCP subset:")
print(invalid_hcp_counts["subject"].unique())

Example: inspect one subject with inconsistencies

In [ ]:
# Filter rows for a specific subject
filtered = aug_track[aug_track["subject"] == "4_S_5005"]

In [ ]:
filtered[["subject", "augmentation", "missing_hcps"]].sort_values("augmentation").reset_index()

#### Folder

Check if augmentations were actually created

In [ ]:
# List all subfolders
subfolders = [f.path for f in os.scandir(fcmaps_augmented_dir) if f.is_dir()]
print(f"Total subfolders (Patients): {len(subfolders)}\n")

count_problems = 0
# Check each subfolder
for folder in subfolders:
    # List only files (ignore subdirectories)
    files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    num_files = len(files)

    # Only print if not exactly 10
    if num_files != 10:
        print(f"[WARNING] {folder} --> {num_files} files (expected 10)")
        count_problems += 1

if count_problems == 0:
    print("All folders contain exactly 10 files.")
else:
    print(f"Total folders with problems: {count_problems}")

Check that each subject has 10 unique augmentations

In [ ]:
problems = []

# Loop over subject folders
for subject_dir in os.listdir(fcmaps_augmented_dir):
    subject_path = os.path.join(fcmaps_augmented_dir, subject_dir)
    if os.path.isdir(subject_path):
        files = [f for f in os.listdir(subject_path) if f.endswith(".nii.gz")]

        # Extract augmentation indices from filenames
        found_aug = sorted([
            int(f.split("aug")[-1].split(".")[0])
            for f in files
            if "aug" in f and f.split("aug")[-1].split(".")[0].isdigit()
        ])

        expected = list(range(1, 11))
        if found_aug != expected:
            problems.append((subject_dir, found_aug))

# Final report
if problems:
    print("Problems found in the following subjects:")
    for subj, augs in problems:
        print(f"[PROBLEM] {subj} has augmentations: {augs}")
else:
    print("All folders contain augmentations from 1 to 10 correctly.")


## Comparing HCPs and SCA files missing

HCP missing as SCA files in the dataset (Lorenzo folder) and here

In [ ]:
import pandas as pd

# Load the CSVs
sca_missing_df = pd.read_csv(path_sca_missing)
aug_track = pd.read_csv(csv_aug_path)

# Variables
n_total_hcp = 173
results = []
unexpected_missing_total = set()

# Loop through subjects
for subject in sca_missing_df["subject"].unique():
    # From missing_SCA_files
    row_sca = sca_missing_df[sca_missing_df["subject"] == subject]
    sca_missing = set(row_sca.iloc[0]["SCA_files_missing"].split(","))
    count_missing = len(sca_missing)

    # From aug_tracking
    sub_aug = aug_track[aug_track["subject"] == subject]
    expected_hcp = set()
    for row in sub_aug["missing_hcps"]:
        hcp_ids = row.strip().strip('"').split(",")
        expected_hcp.update(hcp_ids)

    # Intersections
    expected_and_missing = sca_missing.intersection(expected_hcp)
    unexpected_and_missing = sca_missing - expected_hcp
    unexpected_missing_total.update(unexpected_and_missing)

    # Subject output
    results.append(f"Soggetto {subject}:\n")
    results.append(f"- Missing SCA files: {count_missing}\n")
    results.append(f"- Used SCA files:  {n_total_hcp - count_missing}\n")
    results.append(f"- File attesi nell'augmentation e mancanti come SCA: {', '.join(sorted(expected_and_missing))}\n")
    results.append(f"- File NON attesi nell'augmentation e mancanti come SCA: {', '.join(sorted(unexpected_and_missing))}\n\n")

# Final section
results.append("Unique list of NON expected SCA-missing files:\n")
results.append(f"{', '.join(sorted(unexpected_missing_total))}\n")

if unexpected_missing_total == excluded_hcp:
    results.append(f"----> The NON expected and missing SCA files match those excluded from augmentation.\n")

# Write report to file
with open(path_final_report, "w") as f:
    f.writelines(results)
